In [1]:
import torch
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
m="distilbert-base-uncased"
from datasets import load_dataset

emo=load_dataset("emotion")

E:\anaconda\envs\prac\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
E:\anaconda\envs\prac\lib\site-packages\datasets\load.py:1491: FutureWarning: The repository for emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [2]:
from transformers import AutoTokenizer

model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)
emotions_encoded = emo.map(tokenize, batched=True, batch_size=None)


E:\anaconda\envs\prac\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [3]:
from transformers import AutoModelForSequenceClassification

num_labels=6 

model=(AutoModelForSequenceClassification.from_pretrained(m,num_labels=num_labels).to(device))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'pre_classifier.we

In [4]:
from sklearn.metrics import accuracy_score,f1_score

def compute_metrics(pred):  #예측한 결과에 정확도랑 f1 score를 추가해주는 함수
    labels=pred.label_ids
    preds=pred.predictions.argmax(-1)
    f1=f1_score(labels,preds,average="weighted")
    acc=accuracy_score(labels,preds)
    return {"accuracy":acc,"f1":f1}

In [12]:
from huggingface_hub import notebook_login
notebook_login()

In [13]:
from transformers import Trainer, TrainingArguments

batch_size = 64
logging_steps = len(emotions_encoded["train"]) // batch_size
model_name = f"{model_ckpt}-finetuned-emotion-0605"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=2, 
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=True,
                                  save_strategy="epoch",
                                  load_best_model_at_end=True,
                                  log_level="error")

In [14]:
from transformers import Trainer

trainer = Trainer(model=model, args=training_args,
                  train_dataset=emotions_encoded["train"],
                  eval_dataset=emotions_encoded["validation"],
                  compute_metrics=compute_metrics,
                  tokenizer=tokenizer)
trainer.train()

E:\anaconda\envs\prac\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
C:\Users\Owner\jupyter\chapter 2\distilbert-base-uncased-finetuned-emotion-0605 is already a clone of https://huggingface.co/kwkwkwkwpark/distilbert-base-uncased-finetuned-emotion-0605. Make sure you pull the latest changes with `repo.git_pull()`.
E:\anaconda\envs\prac\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_w

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.176100,0.180491,0.931000,0.930791
2,0.112200,0.163002,0.937500,0.937700


TrainOutput(global_step=500, training_loss=0.1441614761352539, metrics={'train_runtime': 111.7977, 'train_samples_per_second': 286.231, 'train_steps_per_second': 4.472, 'total_flos': 720342861696000.0, 'train_loss': 0.1441614761352539, 'epoch': 2.0})

In [15]:
pre=trainer.predict(emotions_encoded["validation"])

In [16]:
pre.metrics

{'test_loss': 0.16300195455551147,
 'test_accuracy': 0.9375,
 'test_f1': 0.9376998527208639,
 'test_runtime': 1.6048,
 'test_samples_per_second': 1246.253,
 'test_steps_per_second': 19.94}

In [18]:
import numpy as np
y_pred=np.argmax(pre.predictions,axis=1)
y_valid = np.array(emotions_hidden["validation"]["label"])  # 내가 단위별로 코드를 나눠서 여기서 에러가 남 validataion 데이터 정답값을 가져옴
labels = emo["train"].features["label"].names  
plot_confusion_matrix(y_pred,y_valid,labels)  #가져온 데이터를 기준으로 오차 matrix를 보여줌 

NameError: name 'emotions_hidden' is not defined